In [1]:
import sys, plotly
print("python:", sys.executable)
print("plotly:", plotly.__version__)
try:
    import nbformat
    print("nbformat:", nbformat.__version__)
except Exception as e:
    print("nbformat import error:", e)
import plotly.io as pio
print("renderer:", pio.renderers.default)

python: c:\Users\Ivan\roboAI\.venv\Scripts\python.exe
plotly: 6.3.1
nbformat: 5.10.4
renderer: vscode


In [2]:
import json, os
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "browser"

LOG_DIR = Path("../data/logs")
files = sorted(LOG_DIR.glob("run_*.json"))
assert files, f"No logs found in {LOG_DIR}"
with open(files[-1], "r", encoding="utf-8") as f:
    data = json.load(f)

events = data.get("events", [])
df = pd.json_normalize(events)
df["t0"] = df["t"] - df["t"].min() if "t" in df.columns else range(len(df))

# Path (x,y) from spa_tick
need_cols = {"t0","x","y","left_cmd","right_cmd"}
path_df = df[df["op"] == "spa_tick"].copy()
missing = need_cols - set(path_df.columns)
if missing:
    print("Heads up: missing columns in spa_tick:", missing)
if path_df.empty:
    raise RuntimeError("No 'spa_tick' rows found. Make sure you ran the SPA controller (the planner version logs spa_tick each tick).")

# Wheel command time series
fig_cmds = go.Figure()
fig_cmds.add_trace(go.Scatter(x=path_df["t0"], y=path_df.get("left_cmd"),  name="left_cmd"))
fig_cmds.add_trace(go.Scatter(x=path_df["t0"], y=path_df.get("right_cmd"), name="right_cmd"))
fig_cmds.update_layout(title="Wheel commands vs time", xaxis_title="time (s)", yaxis_title="rad/s")
fig_cmds.show()

# Path plot (x,y)
fig_path = px.line(path_df, x="x", y="y", title="Estimated path")
fig_path.update_yaxes(scaleanchor="x", scaleratio=1)
fig_path.show()

# Front obstacle level during forward ticks
if "front" in df.columns:
    front_df = df[df["op"] == "spa_forward_tick"][["t0","front"]]
    if not front_df.empty:
        fig_front = px.line(front_df, x="t0", y="front", title="Front obstacle level (normalized)")
        fig_front.show()

# Also save HTML copies
out_dir = Path("../reports"); out_dir.mkdir(parents=True, exist_ok=True)
fig_cmds.write_html(out_dir / "wheel_cmds.html")
fig_path.write_html(out_dir / "path.html")
print(f"Loaded log: {files[-1].name} — events: {len(df)}")
print(f"Saved HTML to: {out_dir/'wheel_cmds.html'} and {out_dir/'path.html'}")


Loaded log: run_20251005_205040.json — events: 118
Saved HTML to: ..\reports\wheel_cmds.html and ..\reports\path.html


In [ ]:
# Advanced analyzer for notebook
import json, math
from pathlib import Path
from typing import Dict, Any, List
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

LOG_DIR = Path("../data/logs")
REPORTS_DIR = Path("../reports")
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# pick latest run_*.json safely 
files = sorted(LOG_DIR.glob("run_*.json"))
assert files, f"No logs found in {LOG_DIR.resolve()}"
log_path = files[-1]
display(Markdown(f"**Analyzing:** `{log_path.name}`"))

# load & normalize
with log_path.open("r", encoding="utf-8") as f:
    data = json.load(f)
df = pd.json_normalize(data.get("events", []))
assert "op" in df.columns, "Malformed log: missing 'op' column."
df["t0"] = df["t"] - df["t"].min() if "t" in df.columns else np.arange(len(df)) * 0.064

tick = df[df["op"] == "spa_tick"].copy()
assert not tick.empty, "No 'spa_tick' rows found—run the SPA controller path first."
x  = tick["x"].astype(float).to_numpy()
y  = tick["y"].astype(float).to_numpy()
th = tick.get("theta", pd.Series(np.zeros(len(tick)))).astype(float).to_numpy()
t  = tick["t0"].astype(float).to_numpy()

# helpers
def unwrap_angles(th): return np.unwrap(th)
def finite_diff(x, t):
    if len(x) < 2: return np.zeros_like(x)
    dt = np.diff(t, prepend=t[0] + 1e-6); dt[dt==0] = 1e-6
    return np.gradient(x, dt)
def path_length(xs, ys):
    if len(xs) < 2: return 0.0
    dx, dy = np.diff(xs), np.diff(ys)
    return float(np.sum(np.hypot(dx, dy)))
def moving_avg(a, k=7):
    if len(a)==0: return a
    k = max(1, min(k, len(a)))
    w = np.ones(k)/k
    return np.convolve(a, w, mode="same")

# metrics
dist = path_length(x, y)
dur  = float(t[-1]) if len(t) else 0.0
avg_v = dist/dur if dur > 1e-6 else 0.0
th_u = unwrap_angles(th)
dth_dt = finite_diff(th_u, t)
vx, vy = np.gradient(x, t, edge_order=1), np.gradient(y, t, edge_order=1)
speed = moving_avg(np.hypot(vx, vy), k=7)
total_heading_change_deg = float(np.degrees(np.sum(np.abs(np.diff(th_u)))))

turns = int((df["op"] == "turn_start").sum())
wait_total = float(df.loc[df["op"] == "wait_done", "seconds"].sum()) if "seconds" in df.columns else 0.0
front_max = float(df.loc[df["op"] == "spa_forward_tick", "front"].max()) if "front" in df.columns else None

# anomaly flags
stall = False
if len(speed) > 5 and dur > 0:
    low = speed < max(0.02, 0.02 * np.nanmax(speed))
    longest = 0; cur = 0
    for b in low:
        cur = cur + 1 if b else 0
        longest = max(longest, cur)
    dt_est = np.median(np.diff(t)) if len(t) > 1 else 0.064
    stall = (longest * dt_est) >= 1.5

osc = False
if len(dth_dt) > 10:
    sign = np.sign(dth_dt)
    flips = np.sum(np.abs(np.diff(sign)) > 1e-3)
    osc = flips > (0.25 * len(dth_dt))

# events to mark
evt_idx = {
    "turn_start": df.index[df["op"]=="turn_start"].tolist(),
    "scan":       df.index[df["op"]=="scan"].tolist(),
    "wait_done":  df.index[df["op"]=="wait_done"].tolist(),
    "stop":       df.index[df["op"]=="stop"].tolist(),
}

# composite image
stem = log_path.stem
img_path = REPORTS_DIR / f"{stem}_trajectory.png"
sp = speed if len(speed)==len(x) else np.interp(np.linspace(0,1,len(x)), np.linspace(0,1,len(speed)), speed)
spn = (sp - np.nanmin(sp)) / (np.nanmax(sp)-np.nanmin(sp)+1e-9)

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1,1,1)
ax.set_title("Robot trajectory (speed-colored) with events & headings")
ax.set_xlabel("x (m)"); ax.set_ylabel("y (m)"); ax.axis("equal")
# colored trail
for i in range(1, len(x)):
    ax.plot([x[i-1], x[i]], [y[i-1], y[i]], alpha=0.9, linewidth=2,
            color=(spn[i], 0.2, 1.0 - spn[i]))  # gradient
# start/end
ax.scatter([x[0]],[y[0]], marker="o", s=60, label="start")
ax.scatter([x[-1]],[y[-1]], marker="x", s=80, label="end")
# heading arrows (downsampled)
step = max(1, len(x)//30)
u, v = np.cos(th[::step]), np.sin(th[::step])
ax.quiver(x[::step], y[::step], u, v, angles='xy', scale_units='xy', scale=10, alpha=0.5)
# event markers
def _mark(idxs, m, label):
    lab = label
    for idx in idxs:
        j = min(max(idx, 0), len(x)-1)
        ax.scatter([x[j]],[y[j]], marker=m, s=80, label=lab)
        lab = None
_mark(evt_idx["turn_start"], "^", "turn_start")
_mark(evt_idx["scan"], "s", "scan")
_mark(evt_idx["wait_done"], "D", "wait_done")
_mark(evt_idx["stop"], "P", "stop")
ax.legend(loc="best")
fig.tight_layout(); fig.savefig(img_path, dpi=180, bbox_inches="tight"); plt.close(fig)

# paragraph + markdown report
paragraph_bits = [
    f"Over {dur:.1f} s, the robot traveled {dist:.2f} m at an average speed of {avg_v:.2f} m/s.",
    f"It executed ~{turns} turn(s) (total heading change ≈ {total_heading_change_deg:.0f}°).",
]
if wait_total > 0: paragraph_bits.append(f"Total wait time was {wait_total:.1f} s.")
if front_max is not None and not math.isnan(front_max):
    paragraph_bits.append(f"Peak front obstacle level observed was {front_max:.2f} (normalized).")
paragraph_bits.append(f"Trajectory: ({x[0]:.2f}, {y[0]:.2f}) → ({x[-1]:.2f}, {y[-1]:.2f}).")
flags = []
if stall: flags.append("possible stall (prolonged low speed)")
if osc:   flags.append("heading oscillation")
if flags: paragraph_bits.append("Noticed: " + " and ".join(flags) + ".")
paragraph = " ".join(paragraph_bits)

md_path = REPORTS_DIR / f"{stem}_report.md"
md = [
    f"# Run analysis — {log_path.name}",
    f"![trajectory]({img_path.name})",
    "",
    paragraph,
    "",
    "## Key metrics",
    f"- Distance: **{dist:.2f} m**",
    f"- Duration: **{dur:.1f} s**",
    f"- Avg speed: **{avg_v:.2f} m/s**",
    f"- Turns: **{turns}**  (total Δθ ≈ **{total_heading_change_deg:.0f}°**)",
    f"- Wait time: **{wait_total:.1f} s**",
    f"- Max front obstacle level: **{front_max:.2f}**" if front_max is not None and not math.isnan(front_max) else "- Max front obstacle level: n/a",
    f"- Anomalies: **{', '.join(flags) if flags else 'none'}**",
]
md_path.write_text("\n".join(md), encoding="utf-8")

# show inline
display(Markdown(paragraph))
display(Markdown(f"Saved **image** → `{img_path}` & **report** → `{md_path}`"))



**Analyzing:** `run_20251005_205040.json`

Over 4.3 s, the robot traveled 0.11 m at an average speed of 0.03 m/s. It executed ~1 turn(s) (total heading change ≈ 61°). Peak front obstacle level observed was 0.02 (normalized). Trajectory: (0.00, 0.00) → (0.11, -0.00). Noticed: heading oscillation.

Saved **image** → `..\reports\run_20251005_205040_trajectory.png` & **report** → `..\reports\run_20251005_205040_report.md`